In [ ]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor
from haystack.document_stores import ElasticsearchDocumentStore

import torch

print('OK !')

# Embedding Retrieval

In [ ]:
# For EmbeddingRetriever
similarity_type = "cosine"

container_name = "localhost"

er_co = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_bigram_collocation",
                                           similarity=similarity_type, embedding_dim=768)

In [ ]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages = ["en"])

print('OK !')

# Set document_store 

In [ ]:
document_store = er_co

In [ ]:
import spacy
from itertools import combinations

# Set globals
nlp = spacy.load("en_core_web_md")

def pre_process(titles):
    """
    Pre-processes titles by removing stopwords and lemmatizing text.
    :param titles: list of strings, contains target titles,.
    :return: preprocessed_title_docs, list containing pre-processed titles.
    """

    # Preprocess all the titles
    title_docs = [nlp(x) for x in titles]
    preprocessed_title_docs = []
    lemmatized_tokens = []
    for title_doc in title_docs:
        for token in title_doc:
            if not token.is_stop:
                lemmatized_tokens.append(token.lemma_)
        preprocessed_title_docs.append(" ".join(lemmatized_tokens))
        del lemmatized_tokens[
            :
            ]  # empty the lemmatized tokens list as the code moves onto a new title

    return preprocessed_title_docs

def similarity_filter(titles):
    """
    Recursively check if titles pass a similarity filter.
    :param titles: list of strings, contains titles.
    If the function finds titles that fail the similarity test, the above param will be the function output.
    :return: this method upon itself unless there are no similar titles; in that case the feed that was passed
    in is returned.
    """

    # Preprocess titles
    preprocessed_title_docs = pre_process(titles)

    # Remove similar titles
    all_summary_pairs = list(combinations(preprocessed_title_docs, 2))
    similar_titles = []
    for pair in all_summary_pairs:
        title1 = nlp(pair[0])
        title2 = nlp(pair[1])
        similarity = title1.similarity(title2)
        if similarity > 0.8:
            similar_titles.append(pair)

    titles_to_remove = []
    for a_title in similar_titles:
        # Get the index of the first title in the pair
        index_for_removal = preprocessed_title_docs.index(a_title[0])
        titles_to_remove.append(index_for_removal)

    # Get indices of similar titles and remove them
    similar_title_counts = set(titles_to_remove)
    similar_titles = [
        x[1] for x in enumerate(titles) if x[0] in similar_title_counts
    ]

    # Exit the recursion if there are no longer any similar titles
    if len(similar_title_counts) == 0:
        return titles

    # Continue the recursion if there are still titles to remove
    else:
        # Remove similar titles from the next input
        for title in similar_titles:
            idx = titles.index(title)
            titles.pop(idx)
            
        return similarity_filter(titles)

if __name__ == "__main__":
    your_title_list = ['a title', 'the title']
    similarity_filter(your_title_list)

In [ ]:
er_retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="all-mpnet-base-v1",
   model_format="sentence_transformers",
   use_gpu = True,
)

# Set API Server

In [ ]:
from flask import Flask, request, jsonify
from haystack.pipelines import DocumentSearchPipeline

print('OK !')

In [ ]:
app = Flask(__name__)

@app.route('/haystack_collocation', methods=['POST'])
def haystack_collocation():
    query_list = []
    if request.method == 'POST':
        body = request.get_json()
        retriever_pipe = DocumentSearchPipeline(er_retriever)
        prediction = retriever_pipe.run(
            query=body['query'], params={"Retriever": {"top_k": 24},
                                         "filters": body['filters'],
                                         }
        )
        
        doc_list = [each.__dict__ for each in prediction['documents']]
        prediction['documents'] = doc_list
        suggested_co_list = prediction['documents']
        suggested_co_list = [each['content'] for each in suggested_co_list]
        suggested_co_list = similarity_filter(suggested_co_list)
        return {'suggested_keywords': suggested_co_list}, 201

In [ ]:
app.run(port = 7101)